In [1]:
## lm studioのAPIが生きてるか確認
import requests, pprint
pprint.pp(requests.get("http://localhost:1234/v1/models").json())

{'data': [{'id': 'llama-3-elyza-jp-8b',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'text-embedding-nomic-embed-text-v1.5',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'deepseek-r1-distill-llama-8b',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'llm-jp-3-13b-instruct',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'deepseek-r1-distill-qwen-7b',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'llama-3.2-1b-instruct',
           'object': 'model',
           'owned_by': 'organization_owner'}],
 'object': 'list'}


In [2]:
from openai import OpenAI

model_name = 'llama-3-elyza-jp-8b'

client = OpenAI(
    base_url="http://localhost:1234/v1",  # ← ポートを変えた場合はここも変える
    api_key="lm-studio"                  # 何でも良い
)

response = client.chat.completions.create(
    model=model_name,  # LM Studio に表示される識別子
    messages=[
        {"role": "system", "content": "あなたはIQ1000の超天才科学者です。どんな質問にも答えます。"},
        {"role": "user",   "content": "2050年の民主主義はどうなってるか教えてください"}
    ],
    temperature=0.7,
    stream=False,        # True にすればストリームで chunk が取れる
)
print(response.choices[0].message.content)

2050年の民主主義という未来予測を行うのは非常に困難で、多くの仮定や前提が必要です。しかしながら、現状の趨勢と技術の進化を考慮すると、以下は一つの可能性として考えられます。

2050年には、人工知能(AI)、ブロックチェーン、バイオテクノロジーなどの技術が更に進歩し、民主主義も変革することが予測されます。例えば:

1. 遠隔投票や電子投票の普及により、選挙プロセスはより効率的、公正かつ透明性を高めることが可能です。
2. AIとブロックチェーン技術の組み合わせで、選挙結果の自動監査や不正防止が強化されます。
3. デジタル・民主主義の進展に伴い、有権者はオンライン上で情報を得ることが容易になり、政治家とのコミュニケーション方法も多様化します。
4. バイオテクノロジーの進化により、生体認証やDNA鑑定が選挙プロセスに組み込まれ、有権者確認や投票の真正性を担保することができます。
5. 環境問題や経済的格差など、複雑な社会課題に対応するために、多元的、分散型の民主主義が模索されます。例えば、AIによる意思決定支援や、住民投票の活用などです。

2050年の民主主義は、テクノロジーと政治のより緊密な結びつきを特徴とし、「デジタル・民主主義」あるいは「超民主主義」と呼ばれる新しい形態に進化する可能性があります。


In [ ]:
# MEMO: LM Studioはjsonフォーマットでの出力を強制できないため、生成とパースが容易なXMLで出力させる
from openai import OpenAI
import re

def generate_variation_response(text):
    model_name = 'llama-3-elyza-jp-8b'

    client = OpenAI(
        base_url="http://localhost:1234/v1",  # ← ポートを変えた場合はここも変える
        api_key="lm-studio"                  # 何でも良い
    )

    system_prompt = """あなたは入力されたテキストを改変し、**20個** のバリアントを作成します。
バリアントは、意味を大きく変えないように注意しながら、次の戦略で作りなさい。
- 一つ前のバリアントの文章を参照して改変する
- 名詞や動詞を類義語や別の言葉に置き換える
- 単語を抽象化したり、具体化したりする
- 述語の順序を入れ替える、文章の順序を入れ替える
- 文章を短くする、長くする
- 新たな言葉を追加する
- 意味が通る日本語の文章にする、英文にしないこと

出力は必ず次に示すXML形式で返してください。出力は **20個** あることをちゃんと確認してください。

```xml
<original>{original_text}</original>
<variant_01>{variant_text_01}</variant_01>
<variant_02>{variant_text_02}</variant_02>
...省略
<variant_19>{variant_text_19}</variant_19>
<variant_20>{variant_text_20}</variant_20>
```
"""

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": text}
        ],
        temperature=0.8,
        presence_penalty=1.0,       # 同じトークンの出力の抑制
        frequency_penalty=1.0,      # 同じトークンが何度も出た場合の追加ペナルティ
        stream=False,
    )

    # print(pprint.pformat(response.choices[0].message.content))

    re.compile(r"<variant_\d+>(.*?)</variant_\d+>", re.DOTALL)
    # 生成されたXMLからバリエーションを抽出する
    variants = re.findall(r"<variant_\d+>(.*?)</variant_\d+>", response.choices[0].message.content, re.DOTALL)
    return variants

target_text = "自然災害、特に地震が多い日本で原子力発電所を新設・再稼働をすすめることは、大きな危険を長期にわたり国民に背負わせることとなります。"
variants = generate_variation_response(target_text)
# print(target_text)
for v in variants:
    print(v)

# 重複を確認する
print(len(variants), len(set(variants)))


日本の高頻度な地震災害リスクに対し、原子力発電所の新設や再稼動計画は、将来世代まで巨大な危険を課すという重大問題です。
国土が地震多発地域である日本において、新たに原子力発電所を建設したり既存施設の運転再開を行うことは、長期的な安全性懸念を生じさせる行為と言えます。
自然災害が頻繁に起きる日本で、原子力発電所の新しい建設や稼働停止解除計画は、大きな脅威を長期的に国民に課すことになりかねません。
地震被害を受けやすい島国の日本において、新設もしくは再稼動する原子力発電所のリスクは、将来の安全と安心に深刻な影響を及ぼします。
自然災害が頻繁に起きる国土で新たな原子力発電所建設や再稼働計画を推進することは、大きな危険性を長期的に背負うことと同義です。
日本の地震災害リスクに対して、原子力発電所新設・再稼動は将来世代に巨大な安全懸念を課す行為であるという見解が当たらないとは言えないでしょう。
地震の多い日本で原子力発電所を建て直したり、停止中の施設運転再開は長期にわたり国民に大きな危険性をもたらすという点が問題です。
自然災害多発地域である日本において、新たな原子力発電所建設や既存施設の稼働停止解除計画は、長期的な安全懸念に繋がる行為と言えます。
新しい地震被害リスクを常に抱える日本で原子力発電所建設や再稼動の推進には大きな問題点があり、国民は長期的に安全性懸念を受け入れることになります。
高頻度な地震災害が起きる島国の日本で原子力発電所新設や再稼動を推進することは、大きな脅威を将来世代に課す行為であると言えます。
自然災害の多い国土において、新たな原子力発電所建設や既存施設運転停止解除計画は、長期的な安全懸念に繋がるという重大問題です。
地震被害を受けやすい日本で新しい原子力発電所の建設計画や稼働再開を推進するには大きな危険性があり、国民は将来世代に安全懸念を課すことになります。
地震災害が頻繁に起きる日本で原子力発電所新設や稼働再開の計画推進には大きな問題点があり、国民は長期的に安全性懸念を受け入れることになります。
高頻度な地震災害が起きる日本で原子力発電所建設や再稼動の計画推進には大きな脅威を将来世代に課すという重大問題です。
新しい安全性懸念を持つ原子力発電所建設計画は、地震が多く自然災害リスクの高い日本で稼働再開した既存施設と同様、大きな危険を国民に長期的に課す

In [ ]:
variant_list = []

target_text = "自然災害、特に地震が多い日本で原子力発電所を新設・再稼働をすすめることは、大きな危険を長期にわたり国民に背負わせることとなります。"
variant_list.append(target_text)
for i in range(10):
    variants = generate_variation_response(target_text)
    for v in variants:
        if v in variant_list:
            continue
        
        # アルファベットが3割以上のものは除外する、英語の文章が混じっている可能性があるため
        if len(re.findall(r"[a-zA-Z]", v)) / len(v) >= 0.3:
            continue

        print(v)
        variant_list.append(v)

# export file
with open("variant_list.txt", "w", encoding="utf-8") as f:
    f.write(target_text + "\n")
    f.write("\n".join(variant_list))

日本は頻繁な地震の影響により、原子力発電所の新規建設や既存施設の再起動には深刻なリスクが伴い、将来世代まで危険を抱える事態になる可能性があります。
日本は自然災害大国であり特に地震多発地域であるため原子力発電所新設や再稼働には長期のリスクが伴い、将来世代まで危険を背負わせることになります。
日本は地震活動が活発な国であり原子力発電所の新規建設や既存施設の再起動には長期にわたる安全性確保が必要で、将来世代まで危険を抱える事態になる可能性があります。
地震リスクと原子力発電所は切っても切り離せない問題であり、日本の原子力政策には長期的な視点での安全性確保が求められています。
日本で再び大きな被害をもたらす可能性がある地震に対して、既存の原子力発電所は危険に晒されており、新たな施設建設や稼働には長期的な安全性が保証されていない。
日本では自然災害による被害を最小限にすることが求められています。特に地震多発地域で原子力発電所新設や再稼動は危険な選択肢です。
大規模な天災や地質活動の活発化する日本において、原子力発電所の建設と運転再開は、数十年以上もの間、多くの人々を危険に曝すという重い責任を背負わせることです。
地震や津波などの自然災害が頻繁に発生する日本で原子力発電所の新設と再稼働は、長期的な危険性を国民全体に押し付ける結果となります。
この国で地震や津波による被害が多く報告されているにもかかわらず、原子力発電所の新設と再稼働は大きな長期的リスクを背負わせることになります。
日本における地震活動の高さに加え、津波や台風被害が多く報告されている国で原子力発電所を新設・再稼働することは危険性を長期的に抱える結果となります。
この自然災害が多い日本においては地震の影響による津波や、台風被害などから復興した後でさえも原子力発電所新設と再稼働には危険性を長期にわたり国民全体に押し付けることになります。
日本の地震活動が活発な地域での原子力発電所建設は、数十年以上も大きな安全上の問題点があります。自然災害で多くの被害を受けている国民に長期的リスクを背負わせることです。
地震や津波が日本では特に多いという事実から、原子力発電所新設と再稼働は大きな危険性を抱える結果となります。国民全体の長期的なリスクとして受け止められなければなりません。
地震や自然災害が多く報告されている日本では原子力